In [87]:
import os
import sys
import json
import h5py
import glob
import numpy as np
import tensorflow as tf
import functools

%matplotlib inline
import matplotlib.pyplot as plt

sys.path.append('/om2/user/msaddler/ibm_hearing_aid/ibmHearingAid/multi_gpu/')
import functions_brain_network

sys.path.append('/om2/user/msaddler/python-packages/msutil')
import util_figures_cnn
import util_stimuli
import util_misc


In [88]:
fn_brain_arch = '/saved_models/arch_search_v02_topN/f0_label_192/arch_0302/brain_arch.json'
with open(fn_brain_arch, 'r') as f:
    list_brain_arch = json.load(f)


In [9]:
tf.reset_default_graph()

input_shape = [1, 100, 1000, 1]
input_tensor = tf.placeholder(tf.float32, shape=input_shape, name='input_tensor')
output_tensor, nets = make_brain_net(#functions_brain_network.make_brain_net(
    input_tensor,
    {'f0_label': 700},
    list_brain_arch)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:133: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


In [90]:
def make_brain_net(
    input_tensor,
    n_classes_dict,
    list_brain_arch,
    trainable=True,
    batchnorm_flag=True,
    dropout_flag=True):
    '''
    '''
    list_brain_arch_partial = []
    dict_brain_arch_tensors = {}
    layer_input = input_tensor
    for layer_idx, layer in enumerate(list_brain_arch):
        msg = "Brain architecture should not have multiple layers with the same name."
        assert (layer['args']['name'] not in dict_brain_arch_tensors.keys()), msg
        if layer['layer_type'] == 'tf.layers.conv2d':
            part = functools.partial(conv2d_valid_width_wrapper, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input, trainable=trainable)
        elif layer['layer_type'] == 'tf.nn.relu':
            part = functools.partial(tf.nn.relu, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input)
        elif layer['layer_type'] == 'tf.layers.batch_normalization':
            part = functools.partial(tf.layers.batch_normalization, **layer['args'], fused=False)
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input, training=batchnorm_flag, trainable=trainable)
        elif layer['layer_type'] == 'tf.slice':
            part = functools.partial(tf.slice, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input)
        elif layer['layer_type'] == 'tf.transpose':
            part = functools.partial(tf.transpose, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input)
        elif layer['layer_type'] == 'tfnnresample':
            def tfnnresample_wrapper(tensor_input,
                                     sr_input,
                                     sr_output,
                                     kwargs_nnresample_poly_filter={},
                                     **kwargs):
                """Wrapper designed to ignore layer['args']['name']"""
                return util_stimuli.tfnnresample(
                    tensor_input,
                    sr_input,
                    sr_output,
                    kwargs_nnresample_poly_filter=kwargs_nnresample_poly_filter)
            part = functools.partial(tfnnresample_wrapper, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input)
        elif layer['layer_type'] == 'hpool':
            part = functools.partial(hanning_pooling, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input)
        elif layer['layer_type'] == 'tf.layers.flatten':
            part = functools.partial(tf.layers.flatten, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input)
        elif layer['layer_type'] == 'tf.layers.dense':
            part = functools.partial(tf.layers.dense, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input, trainable=trainable)
        elif layer['layer_type'] == 'tf.layers.dropout':
            part = functools.partial(tf.layers.dropout, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input, training=dropout_flag)
        elif layer['layer_type'] == 'fc_top_classification':
            part = functools.partial(fc_top_classification, n_classes_dict=n_classes_dict, **layer['args'])
            list_brain_arch_partial.append(part)
            dict_brain_arch_tensors[layer['args']['name']] = part(layer_input, trainable=trainable)
        else:
            raise NotImplementedError("layer_type `{}` is not supported".format(layer['layer_type']))
        # Update the input tensor for the next layer
        layer_input = dict_brain_arch_tensors[layer['args']['name']]
    output_tensor = layer_input
    return output_tensor, dict_brain_arch_tensors


def hanning_pooling(
    input_layer,
    strides=2,
    pool_size=8,
    padding='SAME',
    name=None,
    sqrt_window=False,
    normalize=False):
    """
    Add a layer using a hanning kernel for pooling

    Parameters
    ----------
    input_layer : tensorflow tensor
        layer to add hanning pooling to
    strides : int
        proportion downsampling
    top_node : string
        specify the node after which the spectemp filters will be added and used as input for the FFT.
    sqrt_window : boolean
        if true, takes the sqrt of the window (old version), normal window generation has sqrt_window=False
    normalize : boolean
        if true, divide the filter by the sum of its values, so that the smoothed signal is the same amplitude as the original.
    name : False or string
        name for the layer. If false appends "_hpool" to the top_node name

    Returns
    -------
    output_layer : tensorflow tensor
        input_layer with hanning pooling applied
    """
    n_channels = input_layer.get_shape().as_list()[3]
    hanning_window_tensor = make_hanning_kernel_tensor_no_depthwise(n_channels, strides=strides, pool_size=pool_size, sqrt_window=sqrt_window, normalize=normalize, name='%s_hpool_kernel'%name)
    if type(strides)!=list and type(strides)==int:
        strides = [strides, strides] # using square filters
    output_layer = conv2d_for_hpool_valid_width_wrapper(input_layer, filters=hanning_window_tensor, strides=[1, strides[0], strides[1], 1], padding=padding, name=name)
    return output_layer


def make_hanning_kernel_tensor_no_depthwise(
    n_channels,
    strides=2,
    pool_size=8,
    sqrt_window=False,
    normalize=False,
    name=None):
    """
    Make a tensor containing the symmetric 2d hanning kernel to use for the pooling filters
    For strides=2, using pool_size=8 gives a reduction of -24.131545969216841 at 0.25 cycles
    For strides=3, using pool_size=12 gives a reduction of -28.607805482176282 at 1/6 cycles

    This version uses the normal conv2d operation and fills most of the smoothing tensor with zeros. Depthwise convolution
    does not have a second order gradient, and cannot be used with some functions.

    Parameters
    ----------
    n_channels : int
        number of channels to copy the kernel into
    strides : int
        proportion downsampling
    pool_size : int
        how large of a window to use
    sqrt_window : boolean
        if true, takes the sqrt of the window (old version), normal window generation has sqrt_window=False
    normalize : boolean
        if true, divide the filter by the sum of its values, so that the smoothed signal is the same amplitude as the original.
    name : False or string
        name for the layer. If false appends "_hpool" to the top_node name


    Returns
    -------
    hanning_tensor : tensorflow tensor
        tensorflow tensor containing the hanning kernel with size [1 pool_size pool_size n_channels]

    """
    hanning_kernel = make_hanning_kernel(strides=strides,pool_size=pool_size,sqrt_window=sqrt_window, normalize=normalize).astype(np.float32)
    hanning_kernel = np.expand_dims(np.expand_dims(hanning_kernel,0),0) * np.expand_dims(np.expand_dims(np.eye(n_channels),3),3) # [width, width, n_channels, n_channels]
    hanning_tensor = tf.constant(hanning_kernel, dtype=tf.float32, name=name)
    hanning_tensor = tf.transpose(hanning_tensor, [2,3,0,1])
    return hanning_tensor


def make_hanning_kernel(
    strides=2,
    pool_size=8,
    sqrt_window=False,
    normalize=False):
    """
    Make the symmetric 2d hanning kernel to use for the pooling filters
    For strides=2, using pool_size=8 gives a reduction of -24.131545969216841 at 0.25 cycles
    For strides=3, using pool_size=12 gives a reduction of -28.607805482176282 at 1/6 cycles

    Parameters
    ----------
    strides : int
        proportion downsampling
    pool_size : int
        how large of a window to use
    sqrt_window : boolean
        if true, takes the sqrt of the window (old version), normal window generation has sqrt_window=False
    normalize : boolean
        if true, divide the filter by the sum of its values, so that the smoothed signal is the same amplitude as the original.

    Returns
    -------
    two_dimensional_kernel : numpy array
        hanning kernel in 2d to use as a kernel for filtering

    """

    if type(strides)!=list and type(strides)==int:
        strides = [strides, strides] # using square filters
 
    if type(pool_size)!=list and type(pool_size)==int: 
        if pool_size > 1:
            window = 0.5 * (1 - np.cos(2.0 * np.pi * (np.arange(pool_size)) / (pool_size - 1)))
            if sqrt_window: 
                two_dimensional_kernel = np.sqrt(np.outer(window, window))
            else: 
                two_dimensional_kernel = np.outer(window, window)
        else: 
            window = np.ones((1,1))
            two_dimensional_kernel = window # [1x1 kernel]
    elif type(pool_size)==list:
        if pool_size[0] > 1:
            window_h = np.expand_dims(0.5 * (1 - np.cos(2.0 * np.pi * (np.arange(pool_size[0])) / (pool_size[0] - 1))),0)
        else:
            window_h = np.ones((1,1))
        if pool_size[1] > 1:
            window_w = np.expand_dims(0.5 * (1 - np.cos(2.0 * np.pi * (np.arange(pool_size[1])) / (pool_size[1] - 1))),1)
        else:
            window_w = np.ones((1,1))
 
        if sqrt_window:
            two_dimensional_kernel = np.sqrt(np.outer(window_h, window_w))
        else:  
            two_dimensional_kernel = np.outer(window_h, window_w)

    if normalize:
        two_dimensional_kernel = two_dimensional_kernel/(sum(two_dimensional_kernel.ravel()))        
    
    return two_dimensional_kernel


def conv2d_valid_width_wrapper(inputs,kernel_size,strides,padding,**kwargs):
    """
    Wraps tf.layers.conv2d to allow valid convolution across signal width and
    'same' convolution across signal height when padding is set to "valid_time"
    
  Arguments:
    inputs (TF Tensor): Tensor input.
    kernel_size (int or tuple/list): An integer or tuple/list of 2 integers, specifying the
      height and width of the 2D convolution window.
      Can be a single integer to specify the same value for
      all spatial dimensions.
    strides (int or tuple/list) : An integer or tuple/list of 2 integers,
      specifying the strides of the convolution along the height and width.
      Can be a single integer to specify the same value for
      all spatial dimensions.
      Specifying any stride value != 1 is incompatible with specifying
      any `dilation_rate` value != 1.
    padding (string): One of `"valid"`, `"same"`, or `"valid_time"` (case-insensitive).
    kwargs (dictionary): Specifies all other arguments required by
    tf.layers.conv2d. Passes these directly to function without modification.
        See Tensorflow documentation for further details.

  Returns:
      (TF Tensor): Output of tf.layers.conv2d.
    """

    #Collects relvant parameters    
    size=inputs.get_shape()
    filter_height = kernel_size[0]
    in_height = size[1]

    #Calculates according to SAME padding formula
    if (in_height % strides[0] == 0):
        pad_along_height = max(filter_height - strides[0], 0)
    else:
        pad_along_height = max(filter_height - (in_height % strides[0]), 0)
    pad_top = pad_along_height // 2
    pad_bottom = pad_along_height - pad_top

    #Pads signal if VALID_TIME is selected and padding is necessary
    #Otherwise, pass inputs through and allow specified convolutioon
    if pad_along_height == 0 or padding.upper() != 'VALID_TIME':
        padding = 'VALID' if padding.upper() == 'VALID_TIME' else padding
        output_tensor = tf.layers.conv2d(inputs,kernel_size=kernel_size,
                                         strides=strides,padding=padding,
                                         **kwargs)
    else:
        #Pads input tensor and moves conv2d to valid padding
        paddings = tf.constant([[0,0],[pad_top, pad_bottom], [0, 0],[0,0]])
        input_padded = tf.pad(inputs,paddings)
        output_tensor=tf.layers.conv2d(input_padded,kernel_size=kernel_size,
                                       strides=strides, padding="VALID",
                                       **kwargs)
    return output_tensor


def conv2d_for_hpool_valid_width_wrapper(inputs,filters,strides,padding,**kwargs):
    """
    Wraps tf.layers.conv2d to allow valid convolution across signal width and
    'same' convolution across signal height when padding is set to "valid_time"
    
  Arguments:
    inputs (TF Tensor): Tensor input.
    filters (TF Tensor):  Must have the same type as input.
      A 4-D tensor of shape [filter_height, filter_width, in_channels, out_channels]
    strides (int or tuple/list) : An integer or tuple/list of 2 integers,
      specifying the strides of the convolution along the height and width.
      Can be a single integer to specify the same value for
      all spatial dimensions.
      Specifying any stride value != 1 is incompatible with specifying
      any `dilation_rate` value != 1.
    padding (string): One of `"valid"`, `"same"`, or `"valid_time"` (case-insensitive).
    kwargs (dictionary): Specifies all other arguments required by
    tf.layers.conv2d. Passes these directly to function without modification.
        See Tensorflow documentation for further details.

  Returns:
      (TF Tensor): Output of tf.layers.conv2d.
    """

    #Collects relvant parameters    
    size=inputs.get_shape()
    kernel_size = filters.get_shape()
    filter_height = int(kernel_size[0])
    in_height = int(size[1])

    #Calculates according to SAME padding formula
    if (in_height % strides[0] == 0):
        pad_along_height = max(filter_height - strides[0], 0)
    else:
        pad_along_height = max(filter_height - (in_height % strides[0]), 0)
    pad_top = pad_along_height // 2
    pad_bottom = pad_along_height - pad_top

    #Pads signal if VALID_TIME is selected and padding is necessary
    #Otherwise, pass inputs through and allow specified convolutioon
    if pad_along_height == 0 or padding.upper() != 'VALID_TIME':
        padding = 'VALID' if padding.upper() == 'VALID_TIME' else padding
        output_tensor = tf.nn.conv2d(inputs,filter=filters,
                                         strides=strides,padding=padding,
                                         **kwargs)
    else:
        #Pads input tensor and moves conv2d to valid padding
        paddings = tf.constant([[0,0],[pad_top, pad_bottom], [0, 0],[0,0]])
        input_padded = tf.pad(inputs,paddings)
        output_tensor=tf.nn.conv2d(input_padded,filter=filters,
                                       strides=strides, padding="VALID",
                                       **kwargs)
    return output_tensor


def fc_top_classification(input_tensor, n_classes_dict, **kwargs):
    """
    Builds an fc layer at the top of the network for classification, parses n_classes_dict.

    Args
    ----
    input_tensor (tensorflow tensor) : the input layer for each of the added fc layers
    n_classes_dict (dict) : contains the number of classes (number of FC units) for each of the tasks
    kwargs : keyword arguments to pass into tf.layers.dense

    Outputs
    -------
    output_tensor (tensorflow tensor) : an fc layer with the number of classes
 
    """

    assert len(list(n_classes_dict.keys())) == 1, "Multiple tasks specified but only one FC layer can be constructed with 'fc_top_classification', please check network configuration."
    (task_name, task_classes), = n_classes_dict.items()
    output_tensor = tf.layers.dense(input_tensor, units=task_classes, **kwargs)
    return output_tensor


In [109]:
tf.reset_default_graph()

input_shape = [1, 1, 4800, 1]
input_tensor = tf.placeholder(tf.float32, shape=input_shape, name='input_tensor')

list_brain_arch_tmp = [
    {
        'args': {
            'name': 'coch_slice_0',
            'begin': [0, 0, 1600, 0],
            'size': [-1, -1, 3200, -1],
        },
        'layer_type': 'tf.slice'
    },
    {
        'args': {
            'activation': None,
            'dilation_rate': [1, 1],
            'filters': 100,
            'kernel_size': [1, 1601],
            'name': 'coch_conv_0',
            'padding': 'VALID',
            'strides': [1, 1]
        },
        'layer_type': 'tf.layers.conv2d'
    },
    {
        'args': {
            'name': 'coch_transpose_0',
            'perm': [0, 3, 2, 1]
        },
        'layer_type': 'tf.transpose'
    },
    {
        "args": {
            "name": "coch_relu_0"
        },
        "layer_type": "tf.nn.relu"
    },
    {
        "args": {
            "name": "coch_tfnnresample_0",
            "sr_input": 32e3,
            "sr_output": 20e3,
            "kwargs_nnresample_poly_filter": {
                "down": 4,
                "up": 1
            },
        },
        "layer_type": "tfnnresample"
    },
    {
        "args": {
            "name": "coch_relu_1"
        },
        "layer_type": "tf.nn.relu"
    },
]

list_brain_arch_tmp = list_brain_arch_tmp + list_brain_arch

output_tensor, nets = make_brain_net(
    input_tensor,
    {'f0_label': 700},
    list_brain_arch_tmp)

output_tensor, nets


[tfnnresample] interpreting `tensor_input.shape` as [batch, freq, time, channels]
[tfnnresample] using up=1 rather than up=5 for nnresample_poly_filter
[tfnnresample] using down=4 rather than down=8 for nnresample_poly_filter
[tfnnresample] using window_length=8000 for nnresample_poly_filter
[tfnnresample] using cutoff frequency near 4000.0 Hz for anti-aliasing lowpass filter


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:155: DeprecationWarning: Both axis > a.ndim and axis < -a.ndim - 1 are deprecated and will raise an AxisError in the future.


(<tf.Tensor 'fc_top/BiasAdd:0' shape=(1, 700) dtype=float32>,
 {'batch_norm_0': <tf.Tensor 'batch_norm_0/batchnorm/add_1:0' shape=(1, 100, 151, 32) dtype=float32>,
  'batch_norm_1': <tf.Tensor 'batch_norm_1/batchnorm/add_1:0' shape=(1, 82, 21, 64) dtype=float32>,
  'batch_norm_2': <tf.Tensor 'batch_norm_2/batchnorm/add_1:0' shape=(1, 24, 13, 128) dtype=float32>,
  'batch_norm_3': <tf.Tensor 'batch_norm_3/batchnorm/add_1:0' shape=(1, 18, 7, 256) dtype=float32>,
  'batch_norm_4': <tf.Tensor 'batch_norm_4/batchnorm/add_1:0' shape=(1, 5, 5, 512) dtype=float32>,
  'batch_norm_fc_intermediate': <tf.Tensor 'batch_norm_fc_intermediate/batchnorm/add_1:0' shape=(1, 1024) dtype=float32>,
  'coch_conv_0': <tf.Tensor 'coch_conv_0/BiasAdd:0' shape=(1, 1, 1600, 100) dtype=float32>,
  'coch_relu_0': <tf.Tensor 'coch_relu_0:0' shape=(1, 100, 1600, 1) dtype=float32>,
  'coch_relu_1': <tf.Tensor 'coch_relu_1:0' shape=(1, 100, 1000, 1) dtype=float32>,
  'coch_slice_0': <tf.Tensor 'coch_slice_0:0' shape=(1

In [85]:
regex_fn = '/om/scratch/Thu/msaddler/data_pitchnet/PND_v08/noise_TLAS_snr_neg10pos10/PND_*.hdf5'
list_fn = glob.glob(regex_fn)
fn = list_fn[0]

# for k in util_misc.get_hdf5_dataset_key_list(fn):
#     print(k)

with h5py.File(fn, 'r') as f:
    IDX = -50
    idx0 = f['nopad_start_index'][IDX] - f['segment_start_index'][IDX]
    idx1 = f['nopad_end_index'][IDX] - f['segment_end_index'][IDX]
    
    print(f['f0'][IDX], f['nopad_f0_mean'][IDX])
    print(f['stimuli/signal_in_noise'][IDX, idx0:idx1].shape[0] / f['sr'][0])
    print(idx0, idx1)


80.4471 80.4471
0.07
2240 -320


In [107]:
X = np.arange(0, 4800)
X[2240:-320].shape

Y = X[1600:]
Y[800:-800]


array([2400, 2401, 2402, ..., 3997, 3998, 3999])

In [108]:
4800-320

4480